__Business problem__:  Given a two set of X company data, existing employees and employees who lift. 
   -  What are factors that let employees to leave 
   -  redict emplyees that are prone to leave

Import Libraries

In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

## Exploratory Data Analaysis

Read the datasets

In [191]:
# read xlsx file sheets
xlsx_file = pd.ExcelFile("./Hash-Analytic-Python-Analytics-Problem-case-study-1.xlsx")
xlsx_file.sheet_names

['INFO', 'Existing employees', 'Employees who have left', 'Presentation']

In [192]:
# Read Employees sheets and save them into csv file
here = "./"  # current directorty
pattern = "(^Existing)|(^Employees)"
for sheets in xlsx_file.sheet_names:
    if re.search(pattern, sheets):  # extract only the employees sheets
        sheet = pd.read_excel("./Hash-Analytic-Python-Analytics-Problem-case-study-1.xlsx",sheet_name=sheets)
        csv_name = (here + str(sheets) + '.csv').replace(" ", "") # remove spaces from string
        # save the data as csv file
        sheet.to_csv(csv_name, index=None,header=True)
                     

In [194]:
# read our datasets
existing_employees = pd.read_csv('Existingemployees.csv')
lift_employees = pd.read_csv('Employeeswhohaveleft.csv')

In [195]:
print("Number of employee: ", len(existing_employees))
existing_employees.head()

Number of employee:  11428


,Emp ID,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,2001,0.58,0.74,4,215,3,0,0,sales,low
1,2002,0.82,0.67,2,202,3,0,0,sales,low
2,2003,0.45,0.69,5,193,3,0,0,sales,low
3,2004,0.78,0.82,5,247,3,0,0,sales,low
4,2005,0.49,0.60,3,214,2,0,0,sales,low


In [196]:
print("Number of employee lift: ", len(lift_employees))
lift_employees.head()

Number of employee lift:  3571


,Emp ID,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,1,0.38,0.53,2,157,3,0,0,sales,low
1,2,0.80,0.86,5,262,6,0,0,sales,medium
2,3,0.11,0.88,7,272,4,0,0,sales,medium
3,4,0.72,0.87,5,223,5,0,0,sales,low
4,5,0.37,0.52,2,159,3,0,0,sales,low


In [197]:
# remove Emp Id from both Datasets
new_existing_data = existing_employees.drop("Emp ID", axis=1, inplace=False)
new_lift_data = lift_employees.drop("Emp ID", axis=1, inplace=False)

Data Statistics Description

In [198]:
exist_describe = new_existing_data.describe()   # whtiout including the EMP ID attribute
exist_describe

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
count,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000,11428.000000
mean,0.666810,0.715473,3.786664,199.060203,3.380032,0.175009,0.026251
std,0.217104,0.162005,0.979884,45.682731,1.562348,0.379991,0.159889
min,0.120000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000
25%,0.540000,0.580000,3.000000,162.000000,2.000000,0.000000,0.000000
50%,0.690000,0.710000,4.000000,198.000000,3.000000,0.000000,0.000000
75%,0.840000,0.850000,4.000000,238.000000,4.000000,0.000000,0.000000
max,1.000000,1.000000,6.000000,287.000000,10.000000,1.000000,1.000000


In [199]:
lift_describe = new_lift_data.describe() # whitout include the EMP ID attribute
lift_describe

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
count,3571.000000,3571.000000,3571.000000,3571.000000,3571.000000,3571.000000,3571.000000
mean,0.440098,0.718113,3.855503,207.419210,3.876505,0.047326,0.005321
std,0.263933,0.197673,1.818165,61.202825,0.977698,0.212364,0.072759
min,0.090000,0.450000,2.000000,126.000000,2.000000,0.000000,0.000000
25%,0.130000,0.520000,2.000000,146.000000,3.000000,0.000000,0.000000
50%,0.410000,0.790000,4.000000,224.000000,4.000000,0.000000,0.000000
75%,0.730000,0.900000,6.000000,262.000000,5.000000,0.000000,0.000000
max,0.920000,1.000000,7.000000,310.000000,6.000000,1.000000,1.000000


####  What factors/features let Employees lift
numerical features: we will rely on the difference between the mean of each feature from existing and employee who lift, because we don't have the same number of records for both sets

In [200]:
existing_features_mean = exist_describe.iloc[1:2,:]
existing_features_mean

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
mean,0.66681,0.715473,3.786664,199.060203,3.380032,0.175009,0.026251


In [201]:
lift_features_mean = lift_describe.iloc[1:2,:]
lift_features_mean 

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
mean,0.440098,0.718113,3.855503,207.41921,3.876505,0.047326,0.005321


In [202]:
mean_difference = existing_features_mean - lift_features_mean
mean_difference

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years
mean,0.226712,-0.002639,-0.068838,-8.359007,-0.496474,0.127683,0.020931


We can see that some factors of lift employees has a high mean than existing employees and other has less, after some sample 
anlysis we can say that (__satisfaction level, number_project, average_montly_hour, time_spend_company, promotion_last_5years__ ) are numerical features affect employees decision making

__Categorical features : salary and dept__

In [95]:
pivot_table_exist_salary= pd.pivot_table(new_existing_data,index='salary', aggfunc=np.mean)
pivot_table_exist_salary.sort_values(by='satisfaction_level',ascending=False)

,Work_accident,average_montly_hours,last_evaluation,number_project,promotion_last_5years,satisfaction_level,time_spend_company
salary,,,,,,,
high,0.166234,200.348052,0.708606,3.789610,0.062338,0.651879,3.693506
medium,0.168259,199.214272,0.715461,3.788068,0.034315,0.668875,3.435952
low,0.183709,198.617418,0.717028,3.784603,0.010109,0.668103,3.253888


In [97]:
pivot_table_lift_salary= pd.pivot_table(new_lift_data, index='salary', aggfunc=np.mean)
pivot_table_lift_salary.sort_values(by='satisfaction_level',ascending=False)

,Work_accident,average_montly_hours,last_evaluation,number_project,promotion_last_5years,satisfaction_level,time_spend_company
salary,,,,,,,
low,0.043738,206.631215,0.716994,3.836096,0.006446,0.441248,3.874770
medium,0.056188,209.610478,0.724571,3.912680,0.003797,0.438550,3.891420
high,0.000000,193.097561,0.644024,3.451220,0.000000,0.434512,3.682927


form pivot tables we can claim that salary didn't affect employees to make the decision to leave.

In [91]:
pivot_table_exist_dept= pd.pivot_table(new_existing_data,index='dept', aggfunc=np.mean)
pivot_table_exist_dept.sort_values(by=['satisfaction_level'],ascending=False)

,Work_accident,average_montly_hours,last_evaluation,number_project,promotion_last_5years,satisfaction_level,time_spend_company
dept,,,,,,,
IT,0.162474,198.886792,0.713050,3.756813,0.000000,0.677170,3.356394
support,0.187575,199.140980,0.721714,3.783751,0.010155,0.673799,3.213859
marketing,0.195420,198.888550,0.723282,3.720611,0.065649,0.669878,3.480916
sales,0.173065,199.571657,0.709223,3.789187,0.029750,0.668548,3.443698
technical,0.172022,198.471083,0.716609,3.814632,0.012358,0.668319,3.222442
hr,0.156489,199.250000,0.720802,3.702290,0.028626,0.666679,3.192748
product_mng,0.174716,197.765625,0.711435,3.795455,0.000000,0.658466,3.330966
management,0.181818,200.233766,0.723451,3.812616,0.122449,0.654861,4.395176
RandD,0.186186,198.951952,0.706081,3.822823,0.040541,0.653799,3.252252


In [98]:
pivot_table_lift_dept= pd.pivot_table(new_lift_data, index='dept', aggfunc=np.mean)
pivot_table_lift_dept.sort_values(by=['satisfaction_level'],ascending=False)

,Work_accident,average_montly_hours,last_evaluation,number_project,promotion_last_5years,satisfaction_level,time_spend_company
dept,,,,,,,
product_mng,0.045455,207.787879,0.726566,3.848485,0.000000,0.481566,3.989899
marketing,0.049261,200.990148,0.692020,3.581281,0.000000,0.453153,3.857143
support,0.055856,205.636036,0.727315,3.864865,0.005405,0.450901,3.933333
sales,0.045365,205.041420,0.711243,3.736686,0.006903,0.447663,3.812623
hr,0.032558,197.306977,0.679721,3.539535,0.000000,0.433395,3.753488
RandD,0.082645,210.975207,0.745372,4.024793,0.000000,0.432810,4.000000
technical,0.047346,214.183644,0.734132,4.061693,0.004304,0.432525,3.959828
management,0.054945,207.263736,0.727253,4.142857,0.032967,0.422857,3.758242
IT,0.032967,213.849817,0.730037,4.025641,0.010989,0.411868,3.860806


For the departement we can see that some employees lift from dept that others, Thus dept are amngs features of leaving the Company

Decision making features : __satisfaction level, number project, average montly hours, time spend company, promotion last 5 years, dept__

### Data concatenation

In [117]:
# we will add another feature "decision" to both dataset to know if employ exist or lift, decision = 1 if employee exist 0 otherwise
new_existing_data['decision'] = 'existe'
new_lift_data['decision'] = 'lift'

We will concatenate both sets of data with features that most affects employees decision

In [122]:
decision_features = ['satisfaction_level','average_montly_hours','number_project','time_spend_company','promotion_last_5years','dept', 'decision'] 
X_employee_data = pd.concat([new_existing_data[decision_features], new_lift_data[decision_features]])
print('Total number of employees exist and who lift are :', len(X_employee_data))
X_employee_data.sample(frac=1).head(10)  # shuffle and display

Total number of employees exist and who left are : 14999


,satisfaction_level,average_montly_hours,number_project,time_spend_company,promotion_last_5years,dept,decision
1462,0.39,149,2,3,0,sales,left
2395,0.92,193,5,4,0,technical,existe
564,0.83,221,3,2,0,support,existe
1619,0.99,270,4,2,0,hr,existe
8494,0.79,222,5,4,0,technical,existe
5392,0.90,223,3,2,0,IT,existe
8562,0.37,173,5,2,0,accounting,existe
4883,0.80,271,3,4,0,sales,existe
8002,0.90,173,4,3,0,accounting,existe
8513,0.66,268,4,3,0,technical,existe


After Exploratory Data Analysis, and have an idea of features or factors that let people leave company X, we will built a Machine learning classifier to infer automaticaly who will leave.

## Data Preprocessing

Dependent and independent variables separation 

In [130]:
X = X_employee_data.iloc[:,:-1]
Y = X_employee_data.iloc[:,-1]

Categorical variables encoding 

In [144]:
def transformer(X, Y, column_index):
        column_transform = ColumnTransformer(transformers=[('oneHotEncoding',OneHotEncoder(categories='auto'),column_index)], 
                                            remainder='passthrough')
        X = column_transform.fit_transform(X)

        # label Encoding
        label_encode = LabelEncoder()
        Y = label_encode.fit_transform(Y)
        return X,Y

In [145]:
X, Y = transformer(X,Y,[5])

Split data into 70% of train and 30% of train

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.30)

Machine learning classifier: we will test a lot of classifiers (DecisionTree, SVM, KNN)

In [170]:
DTreeClassifier = DecisionTreeClassifier()  # Decision Tree Classifier
SVMClassifier = SVC()                         # SVM Classifier
KNNClassifier = KNeighborsClassifier()       # KNN Classifier

fitting the models to our data

In [175]:
# Decision Tree Classifier
DTreeClassifier.fit(X_train,y_train)
y_pred_DTree = DTreeClassifier.predict(X_test)

# SVM Classifier
SVMClassifier.fit(X_train, y_train)
y_predict_SVM = SVMClassifier.predict(X_test)

# KNN Classifier
KNNClassifier.fit(X_train, y_train)
y_predict_KNN = KNNClassifier.predict(X_test)

print("Classificatin Done !!")

Classificatin Done !!


Generate Classification Reports

In [179]:
# Accurcy score of each classifieer
print("Dicision Tree classification accuracy: {0}%".format(round(accuracy_score(y_test, y_pred_DTree),3)*100))
print("SVM classification accuracy: {0}%".format(round(accuracy_score(y_test, y_predict_SVM),3)*100))
print("KNN Tree classification accuracy: {0}%".format(round(accuracy_score(y_test, y_predict_KNN),3)*100))

Dicision Tree classification accuracy: 97.2%
SVM classification accuracy: 77.7%
KNN Tree classification accuracy: 94.0%


In [177]:
# Decision Tree report
DT_classification_report = classification_report(y_test, y_pred_DTree)
print("Decision Tree Classification report:\n",DT_classification_report)

# SVM report
SVM_classification_report = classification_report(y_test, y_predict_SVM)
print("SVM Classification report:\n",SVM_classification_report)

# KNN report
KNN_classification_report = classification_report(y_test, y_predict_KNN)
print("KNN Classification report:\n",KNN_classification_report)

Decision Tree Classification report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98      3419
           1       0.93      0.96      0.94      1081

    accuracy                           0.97      4500
   macro avg       0.96      0.97      0.96      4500
weighted avg       0.97      0.97      0.97      4500

SVM Classification report:
               precision    recall  f1-score   support

           0       0.77      1.00      0.87      3419
           1       1.00      0.07      0.13      1081

    accuracy                           0.78      4500
   macro avg       0.89      0.54      0.50      4500
weighted avg       0.83      0.78      0.69      4500

KNN Classification report:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96      3419
           1       0.84      0.92      0.88      1081

    accuracy                           0.94      4500
   macro avg       0.91      0.93 

From the reports of classification we can see that Decision is outpreform the SVM and KNN models.

we can increase the accracy of classification eather by fitting more models such as naive baise and LinearRegressoin neural network or by starting from the beginning and apply a features selction method

In [ ]:
# Company_Employees_Decisoin
Imagine you are a data scientist within a company, and HR of the company want to automatacly know what employees will leave and wich ones will stay
This sample project is to predict which employ will prone to leave the company X

Salary is not in :smile: :smile: 

Use machine learinng to predict what employee are prone to leave the company

